SQLite Database Initialization
This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

File and Configuration Setup
Additionally, it creates other essential files and configurations needed for subsequent operations.

In [1]:
!rm auction.db

In [2]:
import sqlite3
import os

db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()

In [3]:
from datetime import datetime
from tqdm import tqdm

file_info = {}
data_dir = 'sample/'

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        filepath = os.path.join(root, filename)
        date = datetime.strptime(filename.split('.')[0], '%Y%m%dT%H')

        file_info[filepath] = date

file_info = {k: v for k, v in sorted(file_info.items(), key=lambda item: item[1])}
filenames = list(file_info.keys())

0it [00:00, ?it/s]
100%|██████████| 24/24 [00:00<00:00, 120410.64it/s]


MySQL Items Data Retrieval
In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

SQLite Database Update
After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.

In [4]:
import json
import os
from datetime import datetime
from tqdm import tqdm
import sqlite3

db_path = 'auction.db'
data_dir = 'sample/'
db = sqlite3.connect(db_path)
cursor = db.cursor()

for i, filepath in tqdm(enumerate(filenames)):
    try:
        data = json.load(open(filepath, "r"))
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading file {filepath}: {e}")
        continue

    auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

    if i == 0:
        auction_ids = []
        auctions_data = []

        for auction in data["auctions"]:
            if auction["id"] not in auction_ids:
                auctions_data.append((auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"]))
                auction_ids.append(auction["id"])

        try:
            cursor.executemany("""
                INSERT INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                VALUES (?, ?, ?, ?, ?, ?)
            """, auctions_data)
            db.commit()
        except sqlite3.Error as err:
            db.rollback()
            print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

    action_events_data = []
    for auction in data["auctions"]:
        action_events_data.append((auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')))
            
    try:
        cursor.executemany("""
            INSERT OR REPLACE INTO ActionEvents (auction_id, record)
            VALUES (?, ?)
        """, action_events_data)
        db.commit()
    except sqlite3.Error as err:
        db.rollback()
        print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

cursor.close()
db.close()

48it [02:03,  2.57s/it]


Auction Data and Items Storage
This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

Data Import from Pandas DataFrame
Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [7]:
import pandas as pd
import mysql.connector
import json
import sqlite3

with open('../data/config.json') as f:
    config = json.load(f)

db_path = 'auction.db'
query = "SELECT * FROM Items"

def import_items():
    try:
        mysql_db = mysql.connector.connect(**config['database'])
    except mysql.connector.Error as err:
        print(err)
        return

    cursor = mysql_db.cursor()

    items = cursor.execute(query)
    items = cursor.fetchall()

    mysql_db.close()
    cursor.close()

    try:
        db = sqlite3.connect(db_path)
        print("Connected to SQLite")
    except sqlite3.Error as err:
        return

    cursor = db.cursor()
    cursor.executemany("""
        INSERT OR REPLACE INTO Items (item_id, item_name, quality, item_level, required_level, item_class, item_subclass, purchase_price_gold, purchase_price_silver, sell_price_gold, sell_price_silver, max_count, is_stackable)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, items)

    db.commit()

    cursor.close()
    db.close()

    print("Inserted items into SQLite: " + str(len(items)))

import_items()

Connected to SQLite
Inserted items into SQLite: 10396


/tmp/ipykernel_9257/401834822.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


It is responsible for connecting to an SQLite database, executing a SQL query involving multiple tables, and retrieving the results. These results are stored in the variable

In [93]:
import sqlite3
import pandas as pd
from transformers import TimeLeftTransformer, MedianCompetitorPriceTransformer, AvgCompetitorPriceTransformer, CompetitorCountTransformer, MinCompetitorPriceTransformer, TopCompetitorPriceTransformer , RelativeDifferencesTransformer

db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
    SELECT
        a.auction_id,
        a.bid / 10000 AS bid_in_gold,
        a.buyout / 10000 AS buyout_in_gold,
        (a.buyout / 10000) / a.quantity AS unit_price,
        a.quantity,
        a.time_left,
        a.item_id,
        i.item_name,
        i.quality,
        i.item_class,
        i.item_subclass,
        i.is_stackable,
        i.purchase_price_gold,
        i.required_level,
        i.item_level,
        i.sell_price_gold,
        MIN(ae.record) AS first_appearance_timestamp,
        strftime('%Y', MIN(ae.record)) AS first_appearance_year,
        strftime('%m', MIN(ae.record)) AS first_appearance_month,
        strftime('%d', MIN(ae.record)) AS first_appearance_day,
        strftime('%H', MIN(ae.record)) AS first_appearance_hour,
        COUNT(*) AS hours_on_sale
    FROM Auctions a
    JOIN ActionEvents ae ON a.auction_id = ae.auction_id
    JOIN Items i ON i.item_id = a.item_id
    GROUP BY a.auction_id
"""

cursor.execute(query)
results = cursor.fetchall()

conn.close()

df = pd.DataFrame(results, columns=[i[0] for i in cursor.description])
df.head()

,auction_id,bid_in_gold,buyout_in_gold,unit_price,quantity,time_left,item_id,item_name,quality,item_class,...,purchase_price_gold,required_level,item_level,sell_price_gold,first_appearance_timestamp,first_appearance_year,first_appearance_month,first_appearance_day,first_appearance_hour,hours_on_sale
0,969819641,0,0,0,1,SHORT,8398,Pattern: Tough Scorpid Gloves,Uncommon,Recipe,...,0,0,45,0,2024-01-10 20:00:00,2024,01,10,20,1
1,969819643,0,0,0,1,SHORT,8398,Pattern: Tough Scorpid Gloves,Uncommon,Recipe,...,0,0,45,0,2024-01-10 20:00:00,2024,01,10,20,1
2,969819653,0,0,0,1,SHORT,8395,Pattern: Tough Scorpid Breastplate,Uncommon,Recipe,...,0,0,44,0,2024-01-10 20:00:00,2024,01,10,20,1
3,969819911,27,28,1,20,SHORT,21840,Bolt of Netherweave,Common,Trade Goods,...,1,0,60,0,2024-01-10 20:00:00,2024,01,10,20,1
4,969819913,27,28,1,20,SHORT,21840,Bolt of Netherweave,Common,Trade Goods,...,1,0,60,0,2024-01-10 20:00:00,2024,01,10,20,1


organize the results of a SQL query into a DataFrame, and then apply some form of preprocessing to that data using functions from a module called preprocess_data.

In [94]:
numerical_columns = [
    'quantity',
    'unit_price',
    'bid_in_gold',
    'buyout_in_gold',
    'time_left',
    'median_buyout_price',
    'median_bid_price',
    'median_unit_price',
    'lowest_competitor_price',
    'avg_competitor_price',
    'std_competitor_price',
    'top_competitor_price',
    'competitor_count',
    'rank_buyout_price',
    'rank_bid_price',
    'rank_unit_price',
    'relative_price_difference',
    'relative_avg_price_difference',
    'relative_buyout_difference',
    'relative_bid_difference',
    'relative_price_to_lowest_competitor',
    'relative_price_to_top_competitor',
    'purchase_price_gold',
    'sell_price_gold',
    'required_level',
    'item_level'
]

categorical_columns_ordinal = [
    'item_id',
    'quality',
    'item_class',
    'item_subclass'
]

categorical_columns_onehot = [
  'is_stackable'
]

columns = numerical_columns + categorical_columns_ordinal + categorical_columns_onehot

print('There are', len(numerical_columns), 'numerical columns')
print('There are', len(categorical_columns_ordinal), 'ordinal categorical columns')
print('There are', len(categorical_columns_onehot), 'one-hot categorical columns')
print('There are', len(columns), 'total columns')

There are 26 numerical columns
There are 4 ordinal categorical columns
There are 1 one-hot categorical columns
There are 31 total columns


In [95]:
from data.transformers import transform_data

X, y = transform_data(df, columns)

loads a trained model from a file, makes predictions on a data set, calculates the RMSE and displays the result

In [96]:
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error

pd.options.display.float_format = '{:.0f}'.format

with open('models/linear_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

predictions = model.predict(X)
df['prediction'] = predictions
df[['item_name', 'item_class', 'unit_price', 'buyout_in_gold', 'hours_on_sale', 'prediction']].head(20)

/home/angel/miniconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,item_name,item_class,unit_price,buyout_in_gold,hours_on_sale,prediction
0,Pattern: Tough Scorpid Gloves,Recipe,0,0,1,11
1,Pattern: Tough Scorpid Gloves,Recipe,0,0,1,11
2,Pattern: Tough Scorpid Breastplate,Recipe,0,0,1,11
3,Bolt of Netherweave,Trade Goods,1,28,1,-94
4,Bolt of Netherweave,Trade Goods,1,28,1,-94
5,Bolt of Netherweave,Trade Goods,1,28,1,-94
6,Khan's Cloak,Armor,19,19,1,11
7,Eternium Ore,Trade Goods,0,2,1,13337
8,Dervish Leggings,Armor,39,39,1,14
9,Dervish Leggings,Armor,39,39,1,14


In [73]:
rmse = mean_squared_error(y, predictions, squared=False)

print("RMSE:", rmse)

RMSE: 2129798.64097029


/home/angel/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
